In [12]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
import yfinance
import json

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"] #inputs에는 {"company_name": "Apple"} 이런식으로 들어옴
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    # return stock.income_stmt #pandas dataframe
    return json.dumps(stock.income_stmt.to_json()) #json string

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json()) #json string


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

# a = get_income_statement({"ticker": "AAPL"})
# print(type(a))
# print(a)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
    model="gpt-4-1106-preview",
    tools=functions,
)

print(assistant)
# id='asst_CtI9FmsHF9VLke9iz8pZ3KVw''

Assistant(id='asst_CtI9FmsHF9VLke9iz8pZ3KVw', created_at=1737851182, description=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', metadata={}, model='gpt-4-1106-preview', name='Investor Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Ticker symbol of the company'}}, 'required': ['ticker']}, strict=False), type='function'), FunctionTool(function=Funct